<a href="https://colab.research.google.com/github/Pankaj-2003/Strategy_Performance_Python/blob/main/Quant_Analyst_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime , timedelta

## Dataframe columns
### Date (datetime64[ns]): The date and time of the trade
### Symbol (string): The ticker symbol of the traded security
### Side (string): Either 'buy' or 'sell'
### Size (float, optional): The number of shares traded (default to 1 if not provided)
### Price (float): The price at which the trade was executed

In [2]:
def getTickerPrice(ticker: str, date: datetime) -> float:
    # This function returns the price of the security 'ticker' at the given 'date'
    # For the purpose of this exercise, assume it returns a random number
    return random.uniform(1, 100)  # Example implementation

In [ ]:
metrics = ["Sharpe" , "max-drawdown" ,  "profit factor" , "total net profit" , "Percent Profitable" , "average win\loss Ratio" , "Expectancy" , "Recovery factor" , "average r multiple" , "ROI"]

In [3]:
def calculate_sharpe(daily_returns, risk_free_rate=0.01):
    average_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()

    # Avoid division by zero
    if std_daily_return == 0:
        return np.inf

    sharpe_ratio = (average_daily_return - risk_free_rate) / std_daily_return
    return sharpe_ratio

In [4]:
def calculate_max_drawdown(daily_returns):
    cumulative_return = (1 + daily_returns).cumprod()
    cumulative_max = cumulative_return.cummax()
    drawdown = cumulative_return - cumulative_max
    max_drawdown = drawdown.min()

    return max_drawdown

In [5]:
def calculate_profit_factor(trades):
    winning_trades = trades[trades['Return'] > 0]
    losing_trades = trades[trades['Return'] < 0]

    total_profit = winning_trades['Return'].sum()
    total_loss = -losing_trades['Return'].sum()  # Negate to get positive loss value

    if total_loss == 0:
        return np.inf if total_profit > 0 else 0  # Handle edge cases

    profit_factor = total_profit / total_loss
    return profit_factor


In [6]:
def percent_profitable(trades):
  winning_trades = trades[trades["Return"] > 0]
  total_trades = len(trades)
  percent_profitable = (len(winning_trades) / total_trades) * 100 if total_trades > 0 else 0
  return percent_profitable

In [7]:
def average_win_loss_ratio(trades):
    average_win = trades[trades["Return"] > 0]["Return"].mean() if len(trades[trades["Return"] > 0]) > 0 else 0
    average_loss = trades[trades["Return"] < 0]["Return"].mean() if len(trades[trades["Return"] < 0]) > 0 else 0

    average_win_loss_ratio = average_win / average_loss if average_loss > 0 else 0
    return average_win_loss_ratio


In [8]:
def expectancy(trades):
  average_profit = trades[trades["Return"] > 0]["Return"].mean() if len(trades[trades["Return"] > 0]) > 0 else 0
  average_loss = trades[trades["Return"] < 0]["Return"].mean() if len(trades[trades["Return"] < 0]) > 0 else 0
  winning_trades = trades[trades["Return"] > 0]
  losing_trades = trades[trades["Return"] < 0]
  total_trades = len(trades)
  win_rate = len(winning_trades) / total_trades if total_trades > 0 else 0
  loss_rate = len(losing_trades) / total_trades if total_trades > 0 else 0

  expectancy = (win_rate * average_profit) - (loss_rate * average_loss)
  return expectancy

In [9]:
def recovery_factor(trades  ,daily_returns):
  total_profit = trades[trades['Return'] > 0]['Return'].sum()
  total_loss = -trades[trades['Return'] < 0]['Return'].sum()  # Negate to get positive loss value
  total_net_profit = total_profit - total_loss
  max_dd = calculate_max_drawdown(daily_returns)
  recovery_factor = total_net_profit / abs(max_dd) if max_dd != 0 else np.inf
  return recovery_factor

In [55]:
def trade_perf(trades : pd.DataFrame) -> pd.Series:
  # if dataframe is empty
  if trades.empty:
    return pd.Series({
            'Sharpe': 0,
            'max-drawdown': 0,
            'profit factor': 0,
            'total net profit': 0,
            'Percent Profitable': 0,
            'average win\loss Ratio': 0,
            'Expectancy': 0,
            'Recovery factor': 0,
            'volatility': 0,
            'ROI': 0})

  # handling missing values

  trades['Size'] = trades['Size'].fillna(1)
  trades = trades[trades['Symbol'].notna()]
  trades = trades[trades['Side'].notna()]

  # seperate way to fill price
  def fill_missing_prices(row):
    if pd.isna(row['Price']):
        row['Price'] = getTickerPrice(row['Symbol'], row['Date'])
    return row
  trades = trades.apply(fill_missing_prices, axis=1)




  # calculating returns
  def calculate_return(row):
    current_price = getTickerPrice(row['Symbol'], row['Date'])
    if row['Side'] == 'buy':
        return (current_price - row['Price']) * row['Size']
    else:
        return (row['Price'] - current_price) * row['Size']

  trades['Return'] = trades.apply(calculate_return, axis=1 )  # Applying the function to each row of the df
  daily_returns = trades.groupby('Date')['Return'].sum() # daily return for sharpe


  # 1- calculating metrics

  sharpe_ratio = calculate_sharpe(daily_returns)
  print(trades)

  # 2 - max_drawdown
  maxdd = calculate_max_drawdown(daily_returns)

  # 3 - profit factor
  profit_factor = calculate_profit_factor(trades)

  # 4 - total net profit
  total_profit = trades[trades['Return'] > 0]['Return'].sum()
  total_loss = -trades[trades['Return'] < 0]['Return'].sum()  # Negate to get positive loss value
  total_net_profit = total_profit - total_loss

  return total_net_profit
  # # 5 - percent profitable
  # perc_profitable = percent_profitable(trades)

  # # 6 - average win/loss ratio
  # avg_win_loss_ratio = average_win_loss_ratio(trades)

  # # 7 - expectancy
  # expectncy = expectancy(trades)

  # # 8 - recovery factor
  # recovery_factr = recovery_factor(trades  ,daily_returns)

  # # 9 - volatility
  # volatility = trades['Return'].std()

  # # 10 - roi
  # total_investement = (trades["Price"]* trades["Size"]).sum()
  # roi = (total_net_profit / total_investement) * 100 if total_investement != 0 else 0


  # metrics = pd.Series({
  #           'Sharpe': sharpe_ratio,
  #           'max-drawdown': maxdd,
  #           'profit factor': profit_factor,
  #           'total net profit': total_net_profit,
  #           'Percent Profitable': percent_profitable,
  #           'average win\loss Ratio': average_win_loss_ratio,
  #           'Expectancy': expectancy,
  #           'Recovery factor': recovery_factor,
  #           'volatility': volatility,
  #           'ROI': roi})

  # return metrics

In [56]:
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
sides = ['buy', 'sell']
base_date = datetime(2024, 7, 15)
dates = [base_date + timedelta(days=i//4) for i in range(20)]
data = {
    'Date': dates,
    'Symbol': [random.choice(symbols) for _ in range(20)],
    'Side': [random.choice(sides) for _ in range(20)],
    'Size': [random.randint(1, 100) for _ in range(20)],
    'Price': [random.uniform(50, 300) for _ in range(20)],
}
trades = pd.DataFrame(data)




In [57]:
# Introduce missing values randomly
for col in ['Symbol', 'Size', 'Price']:
    trades.loc[random.sample(range(len(trades)), k=5), col] = np.nan  # Set 5 random entries in each column to NaN

In [58]:
trades

,Date,Symbol,Side,Size,Price
0,2024-07-15,GOOGL,buy,55.0,192.912056
1,2024-07-15,AAPL,buy,NaN,95.428639
2,2024-07-15,MSFT,sell,NaN,NaN
3,2024-07-15,NaN,buy,NaN,266.043252
4,2024-07-16,MSFT,sell,16.0,225.973270
5,2024-07-16,TSLA,sell,66.0,NaN
6,2024-07-16,NaN,sell,61.0,162.173840
7,2024-07-16,AMZN,sell,38.0,104.574450
8,2024-07-17,TSLA,buy,88.0,279.830325
9,2024-07-17,TSLA,buy,60.0,140.372953


In [59]:
trade_perf(trades)

         Date Symbol  Side  Size       Price        Return
0  2024-07-15  GOOGL   buy  55.0  192.912056  -7950.664546
1  2024-07-15   AAPL   buy   1.0   95.428639    -17.234464
2  2024-07-15   MSFT  sell   1.0   29.244792     15.238464
4  2024-07-16   MSFT  sell  16.0  225.973270   2232.663187
5  2024-07-16   TSLA  sell  66.0   20.086713   1054.811124
7  2024-07-16   AMZN  sell  38.0  104.574450   3028.479147
8  2024-07-17   TSLA   buy  88.0  279.830325 -17707.163438
9  2024-07-17   TSLA   buy  60.0  140.372953  -6596.355072
10 2024-07-17   MSFT  sell   1.0  238.759074    173.323943
11 2024-07-17   AMZN   buy  15.0  256.118831  -2850.818078
12 2024-07-18  GOOGL   buy   9.0  263.536535  -1664.655888
13 2024-07-18   AMZN   buy  83.0  153.336468 -11386.335565
15 2024-07-18   AMZN  sell  29.0   49.860342   -332.355617
16 2024-07-19   MSFT   buy  69.0  183.825763  -8017.600851
17 2024-07-19   TSLA  sell  77.0  253.846095  15476.755061


-34541.91259509739

In [60]:
import pandas as pd

# Create DataFrame
data = {
    'Date': ['2024-07-15', '2024-07-15', '2024-07-15', '2024-07-15', '2024-07-16', '2024-07-16', '2024-07-17', '2024-07-17', '2024-07-18', '2024-07-18', '2024-07-18', '2024-07-18', '2024-07-19', '2024-07-19', '2024-07-19'],
    'Symbol': ['AAPL', 'GOOGL', 'MSFT', 'GOOGL', 'MSFT', 'TSLA', 'MSFT', 'AAPL', 'TSLA', 'AMZN', 'TSLA', 'MSFT', 'AMZN', 'TSLA', 'AMZN'],
    'Side': ['buy', 'sell', 'buy', 'sell', 'buy', 'buy', 'sell', 'buy', 'sell', 'sell', 'sell', 'sell', 'buy', 'sell', 'sell'],
    'Size': [1.0, 56.0, 28.0, 62.0, 13.0, 12.0, 45.0, 99.0, 1.0, 15.0, 38.0, 63.0, 32.0, 69.0, 11.0],
    'Price': [156.603089, 157.090902, 179.453593, 157.722247, 67.071367, 166.694237, 85.846357, 93.838908, 216.430890, 151.210652, 75.278121, 123.137375, 84.823568, 70.453734, 80.702777],
    'Return': [-145.449502, 7652.246745, -3610.434913, 8234.835944, -215.616546, -1896.903284, 2105.841654, -6673.449244, 154.176407, 2096.273115, 55.793179, 7236.881600, 456.326576, 241.838136, 649.640473]
}

df = pd.DataFrame(data)

# Separate profitable and losing trades
profitable_trades = df[df['Return'] > 0]['Return']
losing_trades = df[df['Return'] < 0]['Return']

# Calculate total profit and total loss
total_profit = profitable_trades.sum()
total_loss = abs(losing_trades.sum())  # Use absolute value for loss

# Calculate profit factor
profit_factor = total_profit / total_loss if total_loss != 0 else float('inf')

print(f"Total Profit: ${total_profit:.2f}")
print(f"Total Loss: ${total_loss:.2f}")
print(f"Profit Factor: {profit_factor:.2f}")

# Additional analysis
total_trades = len(df)
profitable_trades_count = len(profitable_trades)
losing_trades_count = len(losing_trades)

print(f"\nTotal Trades: {total_trades}")
print(f"Profitable Trades: {profitable_trades_count}")
print(f"Losing Trades: {losing_trades_count}")
print(f"Win Rate: {profitable_trades_count / total_trades:.2%}")

Total Profit: $28883.85
Total Loss: $12541.85
Profit Factor: 2.30

Total Trades: 15
Profitable Trades: 10
Losing Trades: 5
Win Rate: 66.67%
